In [1]:
import utils
import numpy as np
from tensorflow.keras.models import load_model 
from scipy.stats import entropy


2023-10-05 15:13:41.883633: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-05 15:13:41.911843: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 15:13:41.911866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 15:13:41.911884: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 15:13:41.917044: I tensorflow/core/platform/cpu_feature_g

Train baseline models. These will be the reference points to evaluate our solution.

First let's train on the train data as a baseline. 

In [3]:
# Create a new model
model = utils.get_kaggle_model()
model_name = 'baseline_model_trained_on_init_data'

# Load given data
data = utils.get_data()

train_x, train_y = data['train']
prep_train_x, prep_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

# We use early stopping, monitoring val_accuracy with the patience of 8
model, history = utils.train_model(model, prep_train_x,prep_train_y,prep_test_x, prep_test_y,epochs=100, batch_size=64,verbose=1)

print(utils.model_stats(model, prep_test_x, test_y))
model.save(f'{model_name}.keras')
utils.save_history(model_name,history)



Epoch 1/100


2023-10-05 15:14:57.579248: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


16/16 [==============================] - 3s 49ms/step - loss: 3.1017 - accuracy: 0.2110 - val_loss: 2.3591 - val_accuracy: 0.1000
Epoch 2/100
16/16 [==============================] - 0s 32ms/step - loss: 2.4860 - accuracy: 0.2890 - val_loss: 2.4938 - val_accuracy: 0.1048
Epoch 3/100
16/16 [==============================] - 1s 33ms/step - loss: 2.1066 - accuracy: 0.3480 - val_loss: 2.7898 - val_accuracy: 0.1000
Epoch 4/100
16/16 [==============================] - 0s 30ms/step - loss: 2.0594 - accuracy: 0.3730 - val_loss: 2.9153 - val_accuracy: 0.1000
Epoch 5/100
16/16 [==============================] - 0s 30ms/step - loss: 1.8796 - accuracy: 0.3970 - val_loss: 3.0529 - val_accuracy: 0.1000
Epoch 6/100
16/16 [==============================] - 0s 31ms/step - loss: 1.8128 - accuracy: 0.4080 - val_loss: 3.5967 - val_accuracy: 0.1000
Epoch 7/100
16/16 [==============================] - 0s 31ms/step - loss: 1.7328 - accuracy: 0.4520 - val_loss: 3.5900 - val_accuracy: 0.1000
Epoch 8/100
16/16 

Train on the whole dataset (initial train plus data pool). 

In [3]:
model = utils.get_fresh_model()
model_name = 'baseline_model_trained_on_all'

data = utils.get_data()

train_x, train_y = data['train']
pool_x, pool_y = data['pool']
train_x += pool_x
train_y += pool_y

utils.print_class_counts(train_y)

prep_train_x, prep_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

model, history = utils.train_model(model, prep_train_x,prep_train_y,prep_test_x, prep_test_y,epochs=100, batch_size=64,verbose=1)

print(utils.model_stats(model, prep_test_x, test_y))
model.save(f'{model_name}.keras')
utils.save_history(model_name,history)


[5000, 5000, 5000, 472, 5000, 5000, 5000, 535, 5000, 453]
Epoch 1/100
570/570 [==============================] - 6s 7ms/step - loss: 1.8206 - accuracy: 0.2976 - val_loss: 2.1038 - val_accuracy: 0.3184
Epoch 2/100
570/570 [==============================] - 4s 6ms/step - loss: 1.4447 - accuracy: 0.4639 - val_loss: 1.9066 - val_accuracy: 0.3865
Epoch 3/100
570/570 [==============================] - 4s 6ms/step - loss: 1.2409 - accuracy: 0.5498 - val_loss: 1.8581 - val_accuracy: 0.4137
Epoch 4/100
570/570 [==============================] - 4s 6ms/step - loss: 1.1200 - accuracy: 0.6018 - val_loss: 1.6552 - val_accuracy: 0.4611
Epoch 5/100
570/570 [==============================] - 4s 7ms/step - loss: 0.9886 - accuracy: 0.6529 - val_loss: 1.4818 - val_accuracy: 0.5067
Epoch 6/100
570/570 [==============================] - 4s 7ms/step - loss: 0.8855 - accuracy: 0.6943 - val_loss: 1.3550 - val_accuracy: 0.5421
Epoch 7/100
570/570 [==============================] - 4s 7ms/step - loss: 0.7956 - 

To utilize the pool data set most, predict them with the baseline model trained on initial training data. Focus on the ones where model is wrong. From them choose the ones where the entropy is the lowest and so they are sure about their (wrong) decision. 

In [4]:
data = utils.get_data()

train_x, train_y = data['train']
prep_train_x, prep_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

pool_x, pool_y = data['pool']
prep_pool_x, prep_pool_y = utils.prepare_data(pool_x, pool_y)

model= load_model('baseline_model_trained_on_init.keras')

new_train_x = prep_train_x.copy()
new_train_y = prep_train_y.copy()

flag_batch_size = 100
total_flag_size = 1000
misclassed = []
min_misclassed = len(pool_y)
for i in range(int(total_flag_size/flag_batch_size)):
    pred_probs_y = model.predict(prep_pool_x)
    
    pred_pool_y = [np.argmax(x) for x in pred_probs_y]
    stats = utils.model_stats(model,prep_pool_x,np.argmax(prep_pool_y, axis=1))
    worst_class = np.argmin(stats['categorical_accuracy_scores'])
    
    misclassed_count = len(np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)))[0])
    
    misclassified_indexes = np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)) & (np.argmax(prep_pool_y, axis=1)==worst_class))[0]

    print(f'Round {i}, Nr of misclassed', misclassed_count)
    misclassed.append(misclassed_count)
    
    if len(misclassified_indexes)> flag_batch_size:
        selected_indexes = []
        
        # select misclassified samples with the lowest entropy, since these are the worst cases\n",
        entropies = entropy(pred_probs_y, axis=1)
        argsorted_entrop = np.argsort(entropies)

        for index in argsorted_entrop:
            if index in misclassified_indexes:
                selected_indexes.append(index)
        selected_indexes = np.array(selected_indexes)[:flag_batch_size]
    else:
        selected_indexes = misclassified_indexes

    new_train_x = np.vstack((new_train_x, prep_pool_x[selected_indexes]))
    new_train_y = np.vstack((new_train_y, prep_pool_y[selected_indexes]))

    prep_pool_x = np.delete(prep_pool_x, selected_indexes, axis=0)
    prep_pool_y = np.delete(prep_pool_y, selected_indexes, axis=0)

    model = utils.get_fresh_model()
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)
    model_name = 'baseline_model_trained_misclassed_worstclass_lowentropy'

print(utils.model_stats(model, prep_test_x, test_y))
print('Class_counts: ', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name,history)


1109/1109 [==============================] - 1s 1ms/step
Round 0, Nr of misclassed 15030
Epoch 1/55


2023-10-05 14:15:33.299379: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_10/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


18/18 [==============================] - 2s 59ms/step - loss: 2.2677 - accuracy: 0.1136 - val_loss: 2.3454 - val_accuracy: 0.1007
Epoch 2/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1778 - accuracy: 0.1336 - val_loss: 2.4133 - val_accuracy: 0.1548
Epoch 3/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1528 - accuracy: 0.1591 - val_loss: 2.3867 - val_accuracy: 0.1798
Epoch 4/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0964 - accuracy: 0.1955 - val_loss: 2.3195 - val_accuracy: 0.1984
Epoch 5/55
18/18 [==============================] - 0s 25ms/step - loss: 2.0214 - accuracy: 0.2527 - val_loss: 2.3001 - val_accuracy: 0.1873
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9651 - accuracy: 0.2682 - val_loss: 2.2982 - val_accuracy: 0.1537
Epoch 7/55
18/18 [==============================] - 0s 25ms/step - loss: 1.9441 - accuracy: 0.2718 - val_loss: 2.1854 - val_accuracy: 0.2013
Epoch 8/55
18/18 [======

2023-10-05 14:16:47.212058: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/dropout_28/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


22/22 [==============================] - 2s 54ms/step - loss: 2.2992 - accuracy: 0.1271 - val_loss: 2.3040 - val_accuracy: 0.1000
Epoch 2/55
22/22 [==============================] - 0s 21ms/step - loss: 2.2820 - accuracy: 0.1486 - val_loss: 2.3009 - val_accuracy: 0.1000
Epoch 3/55
22/22 [==============================] - 0s 21ms/step - loss: 2.2734 - accuracy: 0.1500 - val_loss: 2.2855 - val_accuracy: 0.1209
Epoch 4/55
22/22 [==============================] - 0s 21ms/step - loss: 2.2313 - accuracy: 0.1721 - val_loss: 2.2356 - val_accuracy: 0.1472
Epoch 5/55
22/22 [==============================] - 0s 21ms/step - loss: 2.1377 - accuracy: 0.1979 - val_loss: 2.1408 - val_accuracy: 0.2012
Epoch 6/55
22/22 [==============================] - 0s 21ms/step - loss: 2.0793 - accuracy: 0.2279 - val_loss: 2.2488 - val_accuracy: 0.1601
Epoch 7/55
22/22 [==============================] - 0s 22ms/step - loss: 2.0689 - accuracy: 0.2250 - val_loss: 2.0783 - val_accuracy: 0.2465
Epoch 8/55
22/22 [======

2023-10-05 14:18:05.427526: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_8/dropout_46/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


27/27 [==============================] - 2s 43ms/step - loss: 2.2761 - accuracy: 0.1700 - val_loss: 2.3204 - val_accuracy: 0.1000
Epoch 2/55
27/27 [==============================] - 0s 18ms/step - loss: 2.2453 - accuracy: 0.1894 - val_loss: 2.3152 - val_accuracy: 0.1000
Epoch 3/55
27/27 [==============================] - 0s 18ms/step - loss: 2.1714 - accuracy: 0.2124 - val_loss: 2.2058 - val_accuracy: 0.1481
Epoch 4/55
27/27 [==============================] - 0s 18ms/step - loss: 2.0481 - accuracy: 0.2253 - val_loss: 2.1946 - val_accuracy: 0.1492
Epoch 5/55
27/27 [==============================] - 0s 19ms/step - loss: 1.9969 - accuracy: 0.2347 - val_loss: 2.1236 - val_accuracy: 0.2096
Epoch 6/55
27/27 [==============================] - 0s 19ms/step - loss: 1.9678 - accuracy: 0.2518 - val_loss: 2.1246 - val_accuracy: 0.1833
Epoch 7/55
27/27 [==============================] - 0s 18ms/step - loss: 1.9354 - accuracy: 0.2735 - val_loss: 2.1072 - val_accuracy: 0.2256
Epoch 8/55
27/27 [======

2023-10-05 14:19:20.675951: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_11/dropout_64/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


32/32 [==============================] - 2s 33ms/step - loss: 2.2321 - accuracy: 0.1905 - val_loss: 2.3325 - val_accuracy: 0.1000
Epoch 2/55
32/32 [==============================] - 0s 16ms/step - loss: 2.1876 - accuracy: 0.2025 - val_loss: 2.3585 - val_accuracy: 0.1000
Epoch 3/55
32/32 [==============================] - 1s 16ms/step - loss: 2.1804 - accuracy: 0.2035 - val_loss: 2.3386 - val_accuracy: 0.0992
Epoch 4/55
32/32 [==============================] - 1s 16ms/step - loss: 2.1657 - accuracy: 0.2120 - val_loss: 2.3244 - val_accuracy: 0.1003
Epoch 5/55
32/32 [==============================] - 0s 16ms/step - loss: 2.0780 - accuracy: 0.2360 - val_loss: 2.2049 - val_accuracy: 0.1665
Epoch 6/55
32/32 [==============================] - 1s 16ms/step - loss: 2.0231 - accuracy: 0.2410 - val_loss: 2.1950 - val_accuracy: 0.1781
Epoch 7/55
32/32 [==============================] - 1s 16ms/step - loss: 1.9209 - accuracy: 0.2760 - val_loss: 2.1302 - val_accuracy: 0.1770
Epoch 8/55
32/32 [======

To utilize the pool data set most, predict them with the baseline model trained on initial training data. Focus on the ones where model is wrong. From them choose the ones where the the margin between the highest probability class and the second highest is the highest. In these cases model is more sure about the (wrong) prediction.

In [5]:
data = utils.get_data()

train_x, train_y = data['train']
prep_train_x, prep_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

pool_x, pool_y = data['pool']
prep_pool_x, prep_pool_y = utils.prepare_data(pool_x, pool_y)

model= load_model('baseline_model_trained_on_init.keras')

new_train_x = prep_train_x.copy()
new_train_y = prep_train_y.copy()

flag_batch_size = 100
total_flag_size = 1000
misclassed = []
min_misclassed = len(pool_y)
for i in range(int(total_flag_size/flag_batch_size)):
    pred_probs_y = model.predict(prep_pool_x)
    
    pred_pool_y = [np.argmax(x) for x in pred_probs_y]
    stats = utils.model_stats(model,prep_pool_x,np.argmax(prep_pool_y, axis=1))
    worst_class = np.argmin(stats['categorical_accuracy_scores'])
    
    misclassed_count = len(np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)))[0])
    
    misclassified_indexes = np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)) & (np.argmax(prep_pool_y, axis=1)==worst_class))[0]

    print(f'Round {i}, Nr of misclassed', misclassed_count)
    misclassed.append(misclassed_count)
    
    if len(misclassified_indexes)> flag_batch_size:
        selected_indexes = []
        # select misclassified samples with the highest margin since these are the worst cases
        sorted_probabilities = np.sort(pred_probs_y)
        margins = sorted_probabilities[:,-1]-sorted_probabilities[:,-2]
        argsorted_margins = np.argsort(margins)[::-1]
        for index in argsorted_margins:
            if index in misclassified_indexes:
                selected_indexes.append(index)
        selected_indexes = np.array(selected_indexes)[:flag_batch_size]
    else:
        selected_indexes = misclassified_indexes

    new_train_x = np.vstack((new_train_x, prep_pool_x[selected_indexes]))
    new_train_y = np.vstack((new_train_y, prep_pool_y[selected_indexes]))

    prep_pool_x = np.delete(prep_pool_x, selected_indexes, axis=0)
    prep_pool_y = np.delete(prep_pool_y, selected_indexes, axis=0)

    model = utils.get_fresh_model()
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)
    model_name = 'baseline_model_trained_misclassed_worstclass_highmargin'

print(utils.model_stats(model, prep_test_x, test_y))
print('Class_counts: ', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])

model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name,history)


1109/1109 [==============================] - 1s 1ms/step
Round 0, Nr of misclassed 15030
Epoch 1/55
18/18 [==============================] - 1s 36ms/step - loss: 2.2656 - accuracy: 0.1282 - val_loss: 2.3506 - val_accuracy: 0.1002
Epoch 2/55
18/18 [==============================] - 0s 26ms/step - loss: 2.1781 - accuracy: 0.1409 - val_loss: 2.3789 - val_accuracy: 0.1428
Epoch 3/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1720 - accuracy: 0.1445 - val_loss: 2.3788 - val_accuracy: 0.1700
Epoch 4/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1538 - accuracy: 0.1627 - val_loss: 2.3974 - val_accuracy: 0.1782
Epoch 5/55
18/18 [==============================] - 0s 24ms/step - loss: 2.1273 - accuracy: 0.1864 - val_loss: 2.3620 - val_accuracy: 0.1728
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0381 - accuracy: 0.2264 - val_loss: 2.3035 - val_accuracy: 0.1471
Epoch 7/55
18/18 [==============================] - 0s 24ms/step 

2023-10-05 14:20:44.708129: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_14/dropout_82/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


21/21 [==============================] - 1s 31ms/step - loss: 2.3013 - accuracy: 0.1108 - val_loss: 2.3034 - val_accuracy: 0.1168
Epoch 2/55
21/21 [==============================] - 0s 22ms/step - loss: 2.2984 - accuracy: 0.0992 - val_loss: 2.3023 - val_accuracy: 0.1079
Epoch 3/55
21/21 [==============================] - 0s 22ms/step - loss: 2.2886 - accuracy: 0.1200 - val_loss: 2.2971 - val_accuracy: 0.1483
Epoch 4/55
21/21 [==============================] - 0s 22ms/step - loss: 2.2633 - accuracy: 0.1400 - val_loss: 2.2683 - val_accuracy: 0.1820
Epoch 5/55
21/21 [==============================] - 0s 22ms/step - loss: 2.2127 - accuracy: 0.1785 - val_loss: 2.2276 - val_accuracy: 0.2006
Epoch 6/55
21/21 [==============================] - 0s 22ms/step - loss: 2.1411 - accuracy: 0.2185 - val_loss: 2.1891 - val_accuracy: 0.1627
Epoch 7/55
21/21 [==============================] - 0s 22ms/step - loss: 2.0930 - accuracy: 0.2277 - val_loss: 2.1134 - val_accuracy: 0.2228
Epoch 8/55
21/21 [======

2023-10-05 14:21:55.024915: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_17/dropout_100/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


25/25 [==============================] - 1s 27ms/step - loss: 2.2772 - accuracy: 0.1663 - val_loss: 2.3125 - val_accuracy: 0.1000
Epoch 2/55
25/25 [==============================] - 0s 20ms/step - loss: 2.2458 - accuracy: 0.1912 - val_loss: 2.3089 - val_accuracy: 0.1000
Epoch 3/55
25/25 [==============================] - 0s 19ms/step - loss: 2.2270 - accuracy: 0.1937 - val_loss: 2.2888 - val_accuracy: 0.1000
Epoch 4/55
25/25 [==============================] - 0s 19ms/step - loss: 2.1926 - accuracy: 0.2019 - val_loss: 2.2809 - val_accuracy: 0.1253
Epoch 5/55
25/25 [==============================] - 0s 19ms/step - loss: 2.1338 - accuracy: 0.2113 - val_loss: 2.2036 - val_accuracy: 0.1669
Epoch 6/55
25/25 [==============================] - 0s 20ms/step - loss: 2.0493 - accuracy: 0.2438 - val_loss: 2.1141 - val_accuracy: 0.1755
Epoch 7/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0648 - accuracy: 0.2306 - val_loss: 2.1645 - val_accuracy: 0.1626
Epoch 8/55
25/25 [======

2023-10-05 14:23:10.281245: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_20/dropout_118/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


30/30 [==============================] - 2s 23ms/step - loss: 2.2376 - accuracy: 0.1637 - val_loss: 2.3282 - val_accuracy: 0.1000
Epoch 2/55
30/30 [==============================] - 0s 17ms/step - loss: 2.2108 - accuracy: 0.1679 - val_loss: 2.3283 - val_accuracy: 0.1000
Epoch 3/55
30/30 [==============================] - 0s 17ms/step - loss: 2.1943 - accuracy: 0.1758 - val_loss: 2.2990 - val_accuracy: 0.1022
Epoch 4/55
30/30 [==============================] - 1s 17ms/step - loss: 2.1316 - accuracy: 0.1989 - val_loss: 2.1848 - val_accuracy: 0.1537
Epoch 5/55
30/30 [==============================] - 0s 17ms/step - loss: 2.0437 - accuracy: 0.2132 - val_loss: 2.1148 - val_accuracy: 0.1887
Epoch 6/55
30/30 [==============================] - 0s 17ms/step - loss: 2.0271 - accuracy: 0.2100 - val_loss: 2.1331 - val_accuracy: 0.1832
Epoch 7/55
30/30 [==============================] - 0s 17ms/step - loss: 2.0059 - accuracy: 0.2405 - val_loss: 2.2426 - val_accuracy: 0.1148
Epoch 8/55
30/30 [======


To utilize the pool data set most, predict them with the baseline model trained on initial training data. Focus on the ones where model is wrong. From them choose the ones where the the class probability is the highest. In these cases model is more sure about the (wrong) prediction.

In [6]:
data = utils.get_data()

train_x, train_y = data['train']
prep_train_x, prep_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

pool_x, pool_y = data['pool']
prep_pool_x, prep_pool_y = utils.prepare_data(pool_x, pool_y)

model= load_model('baseline_model_trained_on_init.keras')

new_train_x = prep_train_x.copy()
new_train_y = prep_train_y.copy()

flag_batch_size = 100
total_flag_size = 1000
misclassed = []
min_misclassed = len(pool_y)
for i in range(int(total_flag_size/flag_batch_size)):
    pred_probs_y = model.predict(prep_pool_x)
    
    pred_pool_y = [np.argmax(x) for x in pred_probs_y]
    stats = utils.model_stats(model,prep_pool_x,np.argmax(prep_pool_y, axis=1))
    worst_class = np.argmin(stats['categorical_accuracy_scores'])
    
    misclassed_count = len(np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)))[0])
    
    misclassified_indexes = np.where((np.argmax(prep_pool_y, axis=1)!=np.array(pred_pool_y)) & (np.argmax(prep_pool_y, axis=1)==worst_class))[0]

    print(f'Round {i}, Nr of misclassed', misclassed_count)
    misclassed.append(misclassed_count)
    
    if len(misclassified_indexes)> flag_batch_size:
        selected_indexes = []
        
        # select misclassified samples with the highest probability, since the are the worst
        max_prob_values = [max(x) for x in pred_probs_y]
        argsorted_max_pred_probs = np.argsort(max_prob_values)[::-1]
        for index in argsorted_max_pred_probs:
            if index in misclassified_indexes:
                selected_indexes.append(index)
        selected_indexes = np.array(selected_indexes)[:flag_batch_size]
    else:
        selected_indexes = misclassified_indexes

    new_train_x = np.vstack((new_train_x, prep_pool_x[selected_indexes]))
    new_train_y = np.vstack((new_train_y, prep_pool_y[selected_indexes]))

    prep_pool_x = np.delete(prep_pool_x, selected_indexes, axis=0)
    prep_pool_y = np.delete(prep_pool_y, selected_indexes, axis=0)

    model = utils.get_fresh_model()
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)
    model_name = 'baseline_model_trained_misclassed_worstclass_highprob'

print(utils.model_stats(model, prep_test_x, test_y))
print('Class_counts: ', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])

model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name,history)


1109/1109 [==============================] - 1s 1ms/step
Round 0, Nr of misclassed 15030
Epoch 1/55
18/18 [==============================] - 1s 35ms/step - loss: 2.2642 - accuracy: 0.1209 - val_loss: 2.3428 - val_accuracy: 0.1000
Epoch 2/55
18/18 [==============================] - 0s 26ms/step - loss: 2.1822 - accuracy: 0.1255 - val_loss: 2.3757 - val_accuracy: 0.1391
Epoch 3/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1536 - accuracy: 0.1673 - val_loss: 2.3584 - val_accuracy: 0.1801
Epoch 4/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1168 - accuracy: 0.1745 - val_loss: 2.3430 - val_accuracy: 0.2064
Epoch 5/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0192 - accuracy: 0.2418 - val_loss: 2.3337 - val_accuracy: 0.1177
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0071 - accuracy: 0.2400 - val_loss: 2.3304 - val_accuracy: 0.1421
Epoch 7/55
18/18 [==============================] - 0s 25ms/step 

2023-10-05 14:24:32.024646: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_23/dropout_136/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


19/19 [==============================] - 1s 35ms/step - loss: 2.2639 - accuracy: 0.1133 - val_loss: 2.3388 - val_accuracy: 0.1002
Epoch 2/55
19/19 [==============================] - 0s 24ms/step - loss: 2.2379 - accuracy: 0.1333 - val_loss: 2.3202 - val_accuracy: 0.1538
Epoch 3/55
19/19 [==============================] - 0s 24ms/step - loss: 2.2003 - accuracy: 0.1458 - val_loss: 2.3123 - val_accuracy: 0.1826
Epoch 4/55
19/19 [==============================] - 0s 24ms/step - loss: 2.1409 - accuracy: 0.1992 - val_loss: 2.2473 - val_accuracy: 0.1857
Epoch 5/55
19/19 [==============================] - 0s 24ms/step - loss: 2.0717 - accuracy: 0.2350 - val_loss: 2.2685 - val_accuracy: 0.1600
Epoch 6/55
19/19 [==============================] - 0s 24ms/step - loss: 2.0452 - accuracy: 0.2317 - val_loss: 2.2046 - val_accuracy: 0.1915
Epoch 7/55
19/19 [==============================] - 0s 24ms/step - loss: 1.9776 - accuracy: 0.2517 - val_loss: 2.1767 - val_accuracy: 0.2050
Epoch 8/55
19/19 [======

2023-10-05 14:25:37.341835: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_26/dropout_154/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


24/24 [==============================] - 1s 27ms/step - loss: 2.2939 - accuracy: 0.1420 - val_loss: 2.3119 - val_accuracy: 0.1000
Epoch 2/55
24/24 [==============================] - 0s 20ms/step - loss: 2.2671 - accuracy: 0.1633 - val_loss: 2.3019 - val_accuracy: 0.1000
Epoch 3/55
24/24 [==============================] - 0s 20ms/step - loss: 2.2534 - accuracy: 0.1553 - val_loss: 2.2924 - val_accuracy: 0.1003
Epoch 4/55
24/24 [==============================] - 0s 20ms/step - loss: 2.1931 - accuracy: 0.1773 - val_loss: 2.2253 - val_accuracy: 0.1654
Epoch 5/55
24/24 [==============================] - 0s 20ms/step - loss: 2.1276 - accuracy: 0.1960 - val_loss: 2.1468 - val_accuracy: 0.2161
Epoch 6/55
24/24 [==============================] - 0s 20ms/step - loss: 2.0753 - accuracy: 0.2233 - val_loss: 2.1548 - val_accuracy: 0.1750
Epoch 7/55
24/24 [==============================] - 0s 20ms/step - loss: 2.0582 - accuracy: 0.2213 - val_loss: 2.0789 - val_accuracy: 0.2215
Epoch 8/55
24/24 [======

2023-10-05 14:26:55.045810: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_29/dropout_172/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


29/29 [==============================] - 1s 23ms/step - loss: 2.2688 - accuracy: 0.1656 - val_loss: 2.3193 - val_accuracy: 0.1000
Epoch 2/55
29/29 [==============================] - 0s 17ms/step - loss: 2.2432 - accuracy: 0.1872 - val_loss: 2.3309 - val_accuracy: 0.1000
Epoch 3/55
29/29 [==============================] - 0s 17ms/step - loss: 2.2275 - accuracy: 0.1878 - val_loss: 2.3040 - val_accuracy: 0.1000
Epoch 4/55
29/29 [==============================] - 0s 17ms/step - loss: 2.1816 - accuracy: 0.1978 - val_loss: 2.2608 - val_accuracy: 0.1027
Epoch 5/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0844 - accuracy: 0.1972 - val_loss: 2.1258 - val_accuracy: 0.1637
Epoch 6/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0886 - accuracy: 0.2106 - val_loss: 2.1528 - val_accuracy: 0.1612
Epoch 7/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0466 - accuracy: 0.2156 - val_loss: 2.1770 - val_accuracy: 0.1595
Epoch 8/55
29/29 [======

As the final baseline, choose 1000 samples randomly and train them to the initial training set. This will be lower baseline for judging the methods.

In [7]:
model = utils.get_fresh_model(seed=0)
model_name = 'baseline_model_randomly_sampled_pool'
data = utils.get_data()
np.random.seed = 42

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

pool_x, pool_y = data['pool']
prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

model = utils.get_fresh_model(seed=0)
selected_indices = np.random.choice(len(pool_x), size=1000, replace=False)

new_train_x = np.vstack((prepared_train_x, prepared_pool_x[selected_indices]))
new_train_y = np.vstack((prepared_train_y, prepared_pool_y[selected_indices]))

model, history = utils.train_model(model,new_train_x, new_train_y, prepared_test_x, prepared_test_y, epochs = 100, batch_size=64,verbose=1)

result = model.evaluate(prepared_test_x, prepared_test_y, 128)[1]

model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]

utils.save_history(model_name,history)

print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts: ', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])


Epoch 1/100


2023-10-05 14:28:00.861935: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_33/dropout_196/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


32/32 [==============================] - 1s 22ms/step - loss: 2.2089 - accuracy: 0.1215 - val_loss: 2.4054 - val_accuracy: 0.1000
Epoch 2/100
32/32 [==============================] - 1s 16ms/step - loss: 2.1012 - accuracy: 0.1460 - val_loss: 2.4535 - val_accuracy: 0.1136
Epoch 3/100
32/32 [==============================] - 1s 16ms/step - loss: 1.9747 - accuracy: 0.2455 - val_loss: 2.3571 - val_accuracy: 0.1641
Epoch 4/100
32/32 [==============================] - 0s 16ms/step - loss: 1.8519 - accuracy: 0.2715 - val_loss: 2.2741 - val_accuracy: 0.2117
Epoch 5/100
32/32 [==============================] - 1s 16ms/step - loss: 1.8872 - accuracy: 0.2865 - val_loss: 2.3515 - val_accuracy: 0.1643
Epoch 6/100
32/32 [==============================] - 0s 16ms/step - loss: 1.8650 - accuracy: 0.2935 - val_loss: 2.3250 - val_accuracy: 0.2206
Epoch 7/100
32/32 [==============================] - 1s 16ms/step - loss: 1.7935 - accuracy: 0.3055 - val_loss: 2.3024 - val_accuracy: 0.2435
Epoch 8/100
32/32 

Solution models

Sample pool by Least confidence method in many steps and retrain model from scratch after each step. Least confidence method chooses the samples where the model predictions max is the lowest, so the model is unsure about the decision.

In [8]:
flag_batch_size = 100
total_flag_size = 1000

model = load_model('baseline_model_trained_on_init.keras')

data = utils.get_data()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

pool_x, pool_y = data['pool']
prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

new_train_x = prepared_train_x.copy()
new_train_y = prepared_train_y.copy()


for i in range(int(total_flag_size/flag_batch_size)):
    cnn_pred_probs = model.predict(prepared_pool_x, batch_size=64)
    certainty_scores = np.max(cnn_pred_probs, axis=1)
    
    argsorted_certainty_scores = np.argsort(certainty_scores)
    selected_indices = argsorted_certainty_scores[:flag_batch_size]
    
    new_train_x = np.vstack((new_train_x, prepared_pool_x[selected_indices]))
    new_train_y = np.vstack((new_train_y, prepared_pool_y[selected_indices]))

    prepared_pool_x = np.delete(prepared_pool_x, selected_indices, axis=0)
    prepared_pool_y = np.delete(prepared_pool_y, selected_indices, axis=0)

    model = utils.get_fresh_model()
    
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)

print('Class_counts: ', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])
print(utils.model_stats(model,prepared_test_x, test_y))

model_name = 'model_trained_by_uncertainty_method'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name, history)

555/555 [==============================] - 1s 2ms/step
Epoch 1/55
18/18 [==============================] - 1s 35ms/step - loss: 2.2260 - accuracy: 0.1264 - val_loss: 2.5154 - val_accuracy: 0.1001
Epoch 2/55
18/18 [==============================] - 0s 24ms/step - loss: 2.1237 - accuracy: 0.1755 - val_loss: 2.4830 - val_accuracy: 0.1025
Epoch 3/55
18/18 [==============================] - 0s 24ms/step - loss: 2.1027 - accuracy: 0.1527 - val_loss: 2.4190 - val_accuracy: 0.1202
Epoch 4/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0777 - accuracy: 0.1818 - val_loss: 2.4867 - val_accuracy: 0.1555
Epoch 5/55
18/18 [==============================] - 0s 25ms/step - loss: 2.0374 - accuracy: 0.2018 - val_loss: 2.3760 - val_accuracy: 0.1816
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9696 - accuracy: 0.2745 - val_loss: 2.3853 - val_accuracy: 0.1251
Epoch 7/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9646 - accuracy: 0.2464 

2023-10-05 14:29:22.034768: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_36/dropout_214/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


21/21 [==============================] - 1s 30ms/step - loss: 2.2218 - accuracy: 0.1423 - val_loss: 2.4069 - val_accuracy: 0.1329
Epoch 2/55
21/21 [==============================] - 0s 22ms/step - loss: 2.1448 - accuracy: 0.1315 - val_loss: 2.4079 - val_accuracy: 0.1523
Epoch 3/55
21/21 [==============================] - 0s 22ms/step - loss: 2.0970 - accuracy: 0.1592 - val_loss: 2.3694 - val_accuracy: 0.1493
Epoch 4/55
21/21 [==============================] - 0s 22ms/step - loss: 2.0426 - accuracy: 0.2215 - val_loss: 2.3618 - val_accuracy: 0.1815
Epoch 5/55
21/21 [==============================] - 0s 21ms/step - loss: 1.9802 - accuracy: 0.2400 - val_loss: 2.3312 - val_accuracy: 0.1963
Epoch 6/55
21/21 [==============================] - 0s 21ms/step - loss: 1.9299 - accuracy: 0.2638 - val_loss: 2.3717 - val_accuracy: 0.1551
Epoch 7/55
21/21 [==============================] - 0s 21ms/step - loss: 1.8972 - accuracy: 0.2685 - val_loss: 2.5299 - val_accuracy: 0.1409
Epoch 8/55
21/21 [======

2023-10-05 14:30:35.582168: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_39/dropout_232/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


25/25 [==============================] - 1s 26ms/step - loss: 2.2571 - accuracy: 0.1381 - val_loss: 2.3456 - val_accuracy: 0.1000
Epoch 2/55
25/25 [==============================] - 0s 19ms/step - loss: 2.1268 - accuracy: 0.1637 - val_loss: 2.4335 - val_accuracy: 0.1081
Epoch 3/55
25/25 [==============================] - 0s 19ms/step - loss: 2.1052 - accuracy: 0.1644 - val_loss: 2.4144 - val_accuracy: 0.1032
Epoch 4/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0683 - accuracy: 0.2025 - val_loss: 2.3928 - val_accuracy: 0.1790
Epoch 5/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0385 - accuracy: 0.2250 - val_loss: 2.3908 - val_accuracy: 0.1628
Epoch 6/55
25/25 [==============================] - 0s 19ms/step - loss: 1.9745 - accuracy: 0.2406 - val_loss: 2.2858 - val_accuracy: 0.2230
Epoch 7/55
25/25 [==============================] - 0s 19ms/step - loss: 1.9501 - accuracy: 0.2650 - val_loss: 2.3265 - val_accuracy: 0.2152
Epoch 8/55
25/25 [======

2023-10-05 14:32:03.385434: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_42/dropout_250/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


30/30 [==============================] - 1s 23ms/step - loss: 2.1996 - accuracy: 0.1447 - val_loss: 2.3657 - val_accuracy: 0.1000
Epoch 2/55
30/30 [==============================] - 0s 17ms/step - loss: 2.1366 - accuracy: 0.1453 - val_loss: 2.3738 - val_accuracy: 0.1018
Epoch 3/55
30/30 [==============================] - 0s 17ms/step - loss: 2.1154 - accuracy: 0.1711 - val_loss: 2.4541 - val_accuracy: 0.1404
Epoch 4/55
30/30 [==============================] - 1s 17ms/step - loss: 2.0906 - accuracy: 0.1821 - val_loss: 2.3796 - val_accuracy: 0.1468
Epoch 5/55
30/30 [==============================] - 0s 17ms/step - loss: 2.0509 - accuracy: 0.2153 - val_loss: 2.3895 - val_accuracy: 0.1025
Epoch 6/55
30/30 [==============================] - 0s 17ms/step - loss: 2.0037 - accuracy: 0.2311 - val_loss: 2.3345 - val_accuracy: 0.1823
Epoch 7/55
30/30 [==============================] - 0s 17ms/step - loss: 1.9562 - accuracy: 0.2537 - val_loss: 2.3032 - val_accuracy: 0.2040
Epoch 8/55
30/30 [======

Sample pool by Least margin method in many steps and retrain model from scratch after each step. Least margin method chooses the samples where difference between the 2 highest class probability is the lowest, so the model is unsure about the decision.


In [9]:
flag_batch_size = 100
total_flag_size = 1000

model = load_model('baseline_model_trained_on_init.keras')

data = utils.get_data()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

pool_x, pool_y = data['pool']
prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

new_train_x = prepared_train_x.copy()
new_train_y = prepared_train_y.copy()


for i in range(int(total_flag_size/flag_batch_size)):
    cnn_pred_probs = model.predict(prepared_pool_x, batch_size=64)

    sorted_probabilities = np.sort(cnn_pred_probs)
    margins = sorted_probabilities[:,-1]-sorted_probabilities[:,-2]
    argsorted_margins = np.argsort(margins)
    
    selected_indices = argsorted_margins[:flag_batch_size]
    
    new_train_x = np.vstack((new_train_x, prepared_pool_x[selected_indices]))
    new_train_y = np.vstack((new_train_y, prepared_pool_y[selected_indices]))

    prepared_pool_x = np.delete(prepared_pool_x, selected_indices, axis=0)
    prepared_pool_y = np.delete(prepared_pool_y, selected_indices, axis=0)

    model = utils.get_fresh_model()
    
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)

print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts:', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])    

model_name = 'model_trained_by_least_margin_sampling_method'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name, history)

555/555 [==============================] - 1s 2ms/step
Epoch 1/55
18/18 [==============================] - 1s 35ms/step - loss: 2.2162 - accuracy: 0.1136 - val_loss: 2.5136 - val_accuracy: 0.1157
Epoch 2/55
18/18 [==============================] - 0s 25ms/step - loss: 2.1048 - accuracy: 0.1745 - val_loss: 2.5228 - val_accuracy: 0.1005
Epoch 3/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0923 - accuracy: 0.1636 - val_loss: 2.4133 - val_accuracy: 0.1403
Epoch 4/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0416 - accuracy: 0.1918 - val_loss: 2.4093 - val_accuracy: 0.1920
Epoch 5/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9731 - accuracy: 0.2282 - val_loss: 2.4090 - val_accuracy: 0.1049
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9723 - accuracy: 0.2255 - val_loss: 2.4156 - val_accuracy: 0.1610
Epoch 7/55
18/18 [==============================] - 0s 25ms/step - loss: 1.8799 - accuracy: 0.2682 

2023-10-05 14:33:22.977721: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_45/dropout_268/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


19/19 [==============================] - 1s 34ms/step - loss: 2.2664 - accuracy: 0.1350 - val_loss: 2.3220 - val_accuracy: 0.1000
Epoch 2/55
19/19 [==============================] - 0s 24ms/step - loss: 2.1290 - accuracy: 0.1550 - val_loss: 2.3689 - val_accuracy: 0.1160
Epoch 3/55
19/19 [==============================] - 0s 23ms/step - loss: 2.0827 - accuracy: 0.1608 - val_loss: 2.4861 - val_accuracy: 0.1030
Epoch 4/55
19/19 [==============================] - 0s 24ms/step - loss: 2.0314 - accuracy: 0.2217 - val_loss: 2.3770 - val_accuracy: 0.1142
Epoch 5/55
19/19 [==============================] - 0s 24ms/step - loss: 1.9650 - accuracy: 0.2458 - val_loss: 2.3725 - val_accuracy: 0.1992
Epoch 6/55
19/19 [==============================] - 0s 24ms/step - loss: 1.8966 - accuracy: 0.2650 - val_loss: 2.4256 - val_accuracy: 0.1687
Epoch 7/55
19/19 [==============================] - 0s 24ms/step - loss: 1.8256 - accuracy: 0.2958 - val_loss: 2.3333 - val_accuracy: 0.2190
Epoch 8/55
19/19 [======

2023-10-05 14:34:37.538716: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_48/dropout_286/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


24/24 [==============================] - 2s 28ms/step - loss: 2.1916 - accuracy: 0.1447 - val_loss: 2.3422 - val_accuracy: 0.1021
Epoch 2/55
24/24 [==============================] - 0s 20ms/step - loss: 2.1186 - accuracy: 0.1707 - val_loss: 2.4376 - val_accuracy: 0.1131
Epoch 3/55
24/24 [==============================] - 0s 20ms/step - loss: 2.0913 - accuracy: 0.1727 - val_loss: 2.3952 - val_accuracy: 0.1442
Epoch 4/55
24/24 [==============================] - 0s 20ms/step - loss: 2.0240 - accuracy: 0.2273 - val_loss: 2.4206 - val_accuracy: 0.1719
Epoch 5/55
24/24 [==============================] - 0s 20ms/step - loss: 1.9227 - accuracy: 0.2340 - val_loss: 2.3876 - val_accuracy: 0.1516
Epoch 6/55
24/24 [==============================] - 0s 20ms/step - loss: 1.9382 - accuracy: 0.2460 - val_loss: 2.3094 - val_accuracy: 0.2192
Epoch 7/55
24/24 [==============================] - 0s 20ms/step - loss: 1.8629 - accuracy: 0.2853 - val_loss: 2.4313 - val_accuracy: 0.1864
Epoch 8/55
24/24 [======

2023-10-05 14:35:58.152586: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_51/dropout_304/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


29/29 [==============================] - 1s 24ms/step - loss: 2.2132 - accuracy: 0.1417 - val_loss: 2.5111 - val_accuracy: 0.1025
Epoch 2/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0931 - accuracy: 0.1683 - val_loss: 2.4353 - val_accuracy: 0.1112
Epoch 3/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0344 - accuracy: 0.1978 - val_loss: 2.3946 - val_accuracy: 0.1063
Epoch 4/55
29/29 [==============================] - 0s 17ms/step - loss: 2.0092 - accuracy: 0.2139 - val_loss: 2.4484 - val_accuracy: 0.1153
Epoch 5/55
29/29 [==============================] - 0s 17ms/step - loss: 1.9622 - accuracy: 0.2256 - val_loss: 2.3698 - val_accuracy: 0.2311
Epoch 6/55
29/29 [==============================] - 0s 16ms/step - loss: 1.9016 - accuracy: 0.2483 - val_loss: 2.5505 - val_accuracy: 0.1277
Epoch 7/55
29/29 [==============================] - 0s 17ms/step - loss: 1.8996 - accuracy: 0.2700 - val_loss: 2.4446 - val_accuracy: 0.2198
Epoch 8/55
29/29 [======

Sample pool by Highest entropy method in many steps and retrain model from scratch after each step. Highest entropy method chooses the samples where the prediction entropy is the highest, so the model is unsure about the decision.


In [10]:
flag_batch_size = 100
total_flag_size = 1000

model = load_model('baseline_model_trained_on_init.keras')

data = utils.get_data()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

pool_x, pool_y = data['pool']
prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

new_train_x = prepared_train_x.copy()
new_train_y = prepared_train_y.copy()


for i in range(int(total_flag_size/flag_batch_size)):
    cnn_pred_probs = model.predict(prepared_pool_x, batch_size=64)

    entropies = entropy(cnn_pred_probs, axis=1)
    argsorted_entrop = np.argsort(entropies)[::-1]
    
    selected_indices = argsorted_entrop[:flag_batch_size]
    
    new_train_x = np.vstack((new_train_x, prepared_pool_x[selected_indices]))
    new_train_y = np.vstack((new_train_y, prepared_pool_y[selected_indices]))

    prepared_pool_x = np.delete(prepared_pool_x, selected_indices, axis=0)
    prepared_pool_y = np.delete(prepared_pool_y, selected_indices, axis=0)

    model = utils.get_fresh_model()
    
    model, history = utils.train_model(model, new_train_x,new_train_y,prep_test_x, prep_test_y,epochs=55, batch_size=64,verbose=1)

print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts:', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])   

model_name = 'model_trained_by_highest_entropy_sampling_method'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name, history)

555/555 [==============================] - 1s 2ms/step
Epoch 1/55


2023-10-05 14:37:26.872378: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_54/dropout_322/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


18/18 [==============================] - 1s 34ms/step - loss: 2.2643 - accuracy: 0.1445 - val_loss: 2.3257 - val_accuracy: 0.1103
Epoch 2/55
18/18 [==============================] - 0s 24ms/step - loss: 2.1222 - accuracy: 0.1455 - val_loss: 2.4488 - val_accuracy: 0.1019
Epoch 3/55
18/18 [==============================] - 0s 24ms/step - loss: 2.1032 - accuracy: 0.1482 - val_loss: 2.4158 - val_accuracy: 0.1083
Epoch 4/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0792 - accuracy: 0.1545 - val_loss: 2.4212 - val_accuracy: 0.1514
Epoch 5/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0556 - accuracy: 0.2182 - val_loss: 2.3812 - val_accuracy: 0.1652
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9769 - accuracy: 0.2600 - val_loss: 2.4317 - val_accuracy: 0.1031
Epoch 7/55
18/18 [==============================] - 0s 25ms/step - loss: 1.9651 - accuracy: 0.2718 - val_loss: 2.3236 - val_accuracy: 0.2112
Epoch 8/55
18/18 [======

2023-10-05 14:38:44.746243: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_57/dropout_340/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


22/22 [==============================] - 1s 29ms/step - loss: 2.2427 - accuracy: 0.1100 - val_loss: 2.4064 - val_accuracy: 0.1000
Epoch 2/55
22/22 [==============================] - 0s 21ms/step - loss: 2.1257 - accuracy: 0.1543 - val_loss: 2.3859 - val_accuracy: 0.1000
Epoch 3/55
22/22 [==============================] - 0s 22ms/step - loss: 2.1027 - accuracy: 0.1614 - val_loss: 2.3752 - val_accuracy: 0.1375
Epoch 4/55
22/22 [==============================] - 0s 21ms/step - loss: 2.0834 - accuracy: 0.1929 - val_loss: 2.3637 - val_accuracy: 0.1401
Epoch 5/55
22/22 [==============================] - 0s 21ms/step - loss: 2.0184 - accuracy: 0.2429 - val_loss: 2.3134 - val_accuracy: 0.1616
Epoch 6/55
22/22 [==============================] - 0s 21ms/step - loss: 1.9713 - accuracy: 0.2300 - val_loss: 2.4930 - val_accuracy: 0.1221
Epoch 7/55
22/22 [==============================] - 0s 21ms/step - loss: 1.9502 - accuracy: 0.2400 - val_loss: 2.4025 - val_accuracy: 0.1712
Epoch 8/55
22/22 [======

2023-10-05 14:40:03.636903: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_60/dropout_358/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


27/27 [==============================] - 2s 27ms/step - loss: 2.2162 - accuracy: 0.1265 - val_loss: 2.3640 - val_accuracy: 0.1087
Epoch 2/55
27/27 [==============================] - 0s 19ms/step - loss: 2.1387 - accuracy: 0.1594 - val_loss: 2.3804 - val_accuracy: 0.1335
Epoch 3/55
27/27 [==============================] - 0s 18ms/step - loss: 2.1199 - accuracy: 0.1782 - val_loss: 2.3846 - val_accuracy: 0.1003
Epoch 4/55
27/27 [==============================] - 0s 19ms/step - loss: 2.0880 - accuracy: 0.1935 - val_loss: 2.3677 - val_accuracy: 0.1218
Epoch 5/55
27/27 [==============================] - 0s 19ms/step - loss: 2.0256 - accuracy: 0.2347 - val_loss: 2.2953 - val_accuracy: 0.1799
Epoch 6/55
27/27 [==============================] - 0s 18ms/step - loss: 2.0072 - accuracy: 0.2494 - val_loss: 2.3642 - val_accuracy: 0.1703
Epoch 7/55
27/27 [==============================] - 0s 18ms/step - loss: 2.0027 - accuracy: 0.2388 - val_loss: 2.3684 - val_accuracy: 0.1601
Epoch 8/55
27/27 [======

2023-10-05 14:41:32.307839: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_63/dropout_376/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


32/32 [==============================] - 2s 22ms/step - loss: 2.2115 - accuracy: 0.1385 - val_loss: 2.3453 - val_accuracy: 0.1201
Epoch 2/55
32/32 [==============================] - 1s 17ms/step - loss: 2.1388 - accuracy: 0.1565 - val_loss: 2.3520 - val_accuracy: 0.0999
Epoch 3/55
32/32 [==============================] - 1s 16ms/step - loss: 2.1231 - accuracy: 0.1695 - val_loss: 2.3464 - val_accuracy: 0.1610
Epoch 4/55
32/32 [==============================] - 1s 17ms/step - loss: 2.0666 - accuracy: 0.1990 - val_loss: 2.2883 - val_accuracy: 0.1896
Epoch 5/55
32/32 [==============================] - 1s 16ms/step - loss: 2.0610 - accuracy: 0.2050 - val_loss: 2.3396 - val_accuracy: 0.1904
Epoch 6/55
32/32 [==============================] - 1s 16ms/step - loss: 2.0261 - accuracy: 0.2180 - val_loss: 2.3049 - val_accuracy: 0.1970
Epoch 7/55
32/32 [==============================] - 1s 16ms/step - loss: 1.9874 - accuracy: 0.2315 - val_loss: 2.4035 - val_accuracy: 0.1632
Epoch 8/55
32/32 [======

Cluster pool data with kmeans, k=10. Calculate the distance for every element from their centroid. Choose the 1000 most distant, since they could be on the border of clusters and in this way can affect the classification more. Retrain model with new dataset

In [11]:
from sklearn.cluster import KMeans


data = utils.get_data()
pool_x, pool_y = data['pool']

prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)
prepared_pool_x = prepared_pool_x.reshape(prepared_pool_x.shape[0], -1)


n_classes = 10
kmeans_model = KMeans(n_clusters=n_classes, n_init=1, max_iter=1)
kmeans_model.fit(prepared_pool_x)

distances = np.array([np.linalg.norm(prepared_pool_x[i]-kmeans_model.cluster_centers_[cluster_id]) for i,cluster_id in enumerate(kmeans_model.labels_)])

argsorted_distances = np.argsort(distances)[::-1]

selected_indices = argsorted_distances[:1000]

model = utils.get_fresh_model()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

new_train_x = np.vstack((prepared_train_x, prepared_pool_x[selected_indices]))
new_train_y = np.vstack((prepared_train_y, prepared_pool_y[selected_indices]))

model, history = utils.train_model(model,new_train_x, new_train_y, prepared_test_x, prepared_test_y, epochs = 100, batch_size=64,verbose=1)

print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts:', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])   

model_name = 'model_trained_by_kmeans_clustered_distant_sampling'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name,history)



Epoch 1/100
32/32 [==============================] - 2s 22ms/step - loss: 2.1746 - accuracy: 0.1630 - val_loss: 2.4152 - val_accuracy: 0.1007
Epoch 2/100
32/32 [==============================] - 1s 17ms/step - loss: 2.0710 - accuracy: 0.2250 - val_loss: 2.4168 - val_accuracy: 0.1494
Epoch 3/100
32/32 [==============================] - 1s 16ms/step - loss: 1.9884 - accuracy: 0.2620 - val_loss: 2.3088 - val_accuracy: 0.2016
Epoch 4/100
32/32 [==============================] - 1s 17ms/step - loss: 1.9125 - accuracy: 0.3010 - val_loss: 2.2762 - val_accuracy: 0.2056
Epoch 5/100
32/32 [==============================] - 1s 17ms/step - loss: 1.8909 - accuracy: 0.3190 - val_loss: 2.2650 - val_accuracy: 0.2195
Epoch 6/100
32/32 [==============================] - 1s 17ms/step - loss: 1.7871 - accuracy: 0.3550 - val_loss: 2.3340 - val_accuracy: 0.1970
Epoch 7/100
32/32 [==============================] - 1s 17ms/step - loss: 1.7569 - accuracy: 0.3595 - val_loss: 2.2742 - val_accuracy: 0.2489
Epoch 

Cluster pool data with kmeans, k=10. Calculate the distance for every element from their centroid. 100 most distant from each cluster, since they could be on the border of clusters and in this way can affect the classification more. Retrain model with new dataset

In [12]:
from sklearn.cluster import KMeans


data = utils.get_data()
pool_x, pool_y = data['pool']

prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)
prepared_pool_x = prepared_pool_x.reshape(prepared_pool_x.shape[0], -1)


n_classes = 10
kmeans_model = KMeans(n_clusters=n_classes, n_init=1, max_iter=1)
kmeans_model.fit(prepared_pool_x)

distances = np.array([np.linalg.norm(prepared_pool_x[i]-kmeans_model.cluster_centers_[cluster_id]) for i,cluster_id in enumerate(kmeans_model.labels_)])


cluster_sample_ids = [np.where(kmeans_model.labels_==i)[0] for i in range(n_classes)]
cluster_sample_distances = []
selected_indices = np.array([])

for i, indices in enumerate(cluster_sample_ids):

    relevant_distances = distances[indices]
    relevant_distances_ids = np.argsort(relevant_distances)[:100]

    selected_indices = np.hstack((selected_indices,indices[relevant_distances_ids])).astype(np.int_)

model = utils.get_fresh_model()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)

prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

new_train_x = np.vstack((prepared_train_x, prepared_pool_x[selected_indices]))
new_train_y = np.vstack((prepared_train_y, prepared_pool_y[selected_indices]))

model, history = utils.train_model(model,new_train_x, new_train_y, prepared_test_x, prepared_test_y, epochs = 100, batch_size=64,verbose=1)
print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts:', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])   


model_name = 'model_trained_by_kmeans_clustered_distant_by_cluster_sampling'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name,history)


Epoch 1/100


2023-10-05 14:42:32.754114: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_65/dropout_388/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


32/32 [==============================] - 2s 23ms/step - loss: 2.1294 - accuracy: 0.1680 - val_loss: 2.4311 - val_accuracy: 0.1000
Epoch 2/100
32/32 [==============================] - 1s 18ms/step - loss: 1.9852 - accuracy: 0.2140 - val_loss: 2.4956 - val_accuracy: 0.1195
Epoch 3/100
32/32 [==============================] - 1s 17ms/step - loss: 1.8734 - accuracy: 0.2950 - val_loss: 2.4600 - val_accuracy: 0.1917
Epoch 4/100
32/32 [==============================] - 1s 17ms/step - loss: 1.7545 - accuracy: 0.3195 - val_loss: 2.5317 - val_accuracy: 0.1598
Epoch 5/100
32/32 [==============================] - 1s 16ms/step - loss: 1.6609 - accuracy: 0.3350 - val_loss: 2.4891 - val_accuracy: 0.2034
Epoch 6/100
32/32 [==============================] - 1s 16ms/step - loss: 1.6371 - accuracy: 0.3510 - val_loss: 2.4548 - val_accuracy: 0.1870
Epoch 7/100
32/32 [==============================] - 1s 16ms/step - loss: 1.6222 - accuracy: 0.3720 - val_loss: 2.3662 - val_accuracy: 0.1929
Epoch 8/100
32/32 

Train KNN classifier on train set. Predict pool with CNN model and KNN. Choose by Least margin method from the ones where the two predictions are different. Retrain models in every step until we reach the 1000 new sample.

In [13]:
from sklearn.neighbors import KNeighborsClassifier

data = utils.get_data()

train_x, train_y = data['train']
prepared_train_x, prepared_train_y = utils.prepare_data(train_x, train_y)
knn_train_x = prepared_train_x.reshape(prepared_train_x.shape[0], -1)

test_x, test_y = data['test']
prepared_test_x, prepared_test_y = utils.prepare_data(test_x, test_y)
knn_test_x =  prepared_test_x.reshape(prepared_test_x.shape[0], -1)


knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(knn_train_x, train_y)

flag_batch_size = 100
total_flag_size = 1000

model = load_model('baseline_model_trained_on_init.keras')

pool_x, pool_y = data['pool']
prepared_pool_x, prepared_pool_y = utils.prepare_data(pool_x, pool_y)

new_train_x = prepared_train_x.copy()
new_train_y = prepared_train_y.copy()


for i in range(int(total_flag_size/flag_batch_size)):

    cnn_pred_probs = model.predict(prepared_pool_x, batch_size=64)
    cnn_predicted_pool_y = [np.argmax(x) for x in cnn_pred_probs]

    knn_pool_x = prepared_pool_x.reshape(prepared_pool_x.shape[0], -1)
    knn_predicted_pool_y = knn_model.predict(knn_pool_x)

    misclassified_indexes = np.where((np.array(knn_predicted_pool_y)!=np.array(cnn_predicted_pool_y)))[0]

    sorted_probabilities = np.sort(cnn_pred_probs)
    margins = sorted_probabilities[:,-1]-sorted_probabilities[:,-2]
    argsorted_margins = np.argsort(margins)
    
    selected_indices = []

    for index in argsorted_margins:
        if index in misclassified_indexes:
            selected_indices.append(index)
    #selected_indices = np.array(selected_indices)[:flag_batch_size]
    selected_indices = np.random.choice(selected_indices, flag_batch_size, replace = False)
    
    new_train_x = np.vstack((new_train_x, prepared_pool_x[selected_indices]))
    new_train_y = np.vstack((new_train_y, prepared_pool_y[selected_indices]))

    new_knn_train_x = new_train_x.reshape(new_train_x.shape[0], -1)
    new_knn_train_y = np.argmax(new_train_y, axis=1)

    prepared_pool_x = np.delete(prepared_pool_x, selected_indices, axis=0)
    prepared_pool_y = np.delete(prepared_pool_y, selected_indices, axis=0)

    model = utils.get_fresh_model()
    
    model, history = utils.train_model(model, new_train_x,new_train_y,prepared_test_x, prepared_test_y,epochs=55, batch_size=64,verbose=1)

    knn_model.fit(new_knn_train_x, new_knn_train_y)



print(utils.model_stats(model,prepared_test_x, test_y))
print('Class_counts:', [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)])   

model_name = 'model_trained_by_knn_cnn_voting_misclassed_sampling'
model.save(f'{model_name}.keras')
history['class_counts'] = [list(np.argmax(new_train_y, axis=1)).count(i) for i in range(10)]
utils.save_history(model_name, history)



555/555 [==============================] - 1s 2ms/step
Epoch 1/55
18/18 [==============================] - 1s 35ms/step - loss: 2.2104 - accuracy: 0.1145 - val_loss: 2.4783 - val_accuracy: 0.1042
Epoch 2/55
18/18 [==============================] - 0s 25ms/step - loss: 2.0978 - accuracy: 0.1527 - val_loss: 2.5670 - val_accuracy: 0.1223
Epoch 3/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0847 - accuracy: 0.1473 - val_loss: 2.4137 - val_accuracy: 0.1913
Epoch 4/55
18/18 [==============================] - 0s 24ms/step - loss: 2.0289 - accuracy: 0.2045 - val_loss: 2.4426 - val_accuracy: 0.1923
Epoch 5/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9380 - accuracy: 0.2800 - val_loss: 2.3955 - val_accuracy: 0.1185
Epoch 6/55
18/18 [==============================] - 0s 24ms/step - loss: 1.8940 - accuracy: 0.2800 - val_loss: 2.6919 - val_accuracy: 0.1043
Epoch 7/55
18/18 [==============================] - 0s 24ms/step - loss: 1.9310 - accuracy: 0.2736 

2023-10-05 14:43:54.185268: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_68/dropout_406/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


21/21 [==============================] - 1s 32ms/step - loss: 2.2044 - accuracy: 0.1354 - val_loss: 2.4466 - val_accuracy: 0.1000
Epoch 2/55
21/21 [==============================] - 0s 22ms/step - loss: 2.1196 - accuracy: 0.1654 - val_loss: 2.4466 - val_accuracy: 0.1167
Epoch 3/55
21/21 [==============================] - 0s 22ms/step - loss: 2.0543 - accuracy: 0.1754 - val_loss: 2.4164 - val_accuracy: 0.1030
Epoch 4/55
21/21 [==============================] - 0s 22ms/step - loss: 1.9914 - accuracy: 0.2292 - val_loss: 2.3755 - val_accuracy: 0.1618
Epoch 5/55
21/21 [==============================] - 0s 22ms/step - loss: 1.8942 - accuracy: 0.2754 - val_loss: 2.5496 - val_accuracy: 0.1187
Epoch 6/55
21/21 [==============================] - 0s 22ms/step - loss: 1.8767 - accuracy: 0.2723 - val_loss: 2.4435 - val_accuracy: 0.1746
Epoch 7/55
21/21 [==============================] - 0s 22ms/step - loss: 1.8445 - accuracy: 0.2838 - val_loss: 2.3129 - val_accuracy: 0.2226
Epoch 8/55
21/21 [======

2023-10-05 14:45:18.057160: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_71/dropout_424/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


25/25 [==============================] - 1s 28ms/step - loss: 2.1948 - accuracy: 0.1450 - val_loss: 2.6523 - val_accuracy: 0.1000
Epoch 2/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0966 - accuracy: 0.1481 - val_loss: 2.5908 - val_accuracy: 0.1287
Epoch 3/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0774 - accuracy: 0.1425 - val_loss: 2.4581 - val_accuracy: 0.1006
Epoch 4/55
25/25 [==============================] - 0s 19ms/step - loss: 2.0456 - accuracy: 0.2069 - val_loss: 2.3961 - val_accuracy: 0.1083
Epoch 5/55
25/25 [==============================] - 0s 20ms/step - loss: 2.0235 - accuracy: 0.2113 - val_loss: 2.4484 - val_accuracy: 0.1474
Epoch 6/55
25/25 [==============================] - 0s 19ms/step - loss: 1.8983 - accuracy: 0.2756 - val_loss: 2.3733 - val_accuracy: 0.2091
Epoch 7/55
25/25 [==============================] - 0s 19ms/step - loss: 1.8994 - accuracy: 0.2688 - val_loss: 2.3343 - val_accuracy: 0.1873
Epoch 8/55
25/25 [======

2023-10-05 14:46:42.570977: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_74/dropout_442/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


30/30 [==============================] - 2s 24ms/step - loss: 2.1602 - accuracy: 0.1447 - val_loss: 2.4190 - val_accuracy: 0.1019
Epoch 2/55
30/30 [==============================] - 1s 18ms/step - loss: 2.0755 - accuracy: 0.1558 - val_loss: 2.4090 - val_accuracy: 0.1107
Epoch 3/55
30/30 [==============================] - 1s 17ms/step - loss: 2.0051 - accuracy: 0.2168 - val_loss: 2.3651 - val_accuracy: 0.1695
Epoch 4/55
30/30 [==============================] - 1s 17ms/step - loss: 1.9355 - accuracy: 0.2611 - val_loss: 2.3483 - val_accuracy: 0.2121
Epoch 5/55
30/30 [==============================] - 0s 17ms/step - loss: 1.8752 - accuracy: 0.2732 - val_loss: 2.3003 - val_accuracy: 0.2081
Epoch 6/55
30/30 [==============================] - 1s 17ms/step - loss: 1.8424 - accuracy: 0.2921 - val_loss: 2.3293 - val_accuracy: 0.2183
Epoch 7/55
30/30 [==============================] - 1s 17ms/step - loss: 1.8119 - accuracy: 0.3042 - val_loss: 2.3156 - val_accuracy: 0.2239
Epoch 8/55
30/30 [======

In [20]:
from pathlib import Path

data = utils.get_data()
test_x, test_y = data['test']
prep_test_x, prep_test_y = utils.prepare_data(test_x, test_y)

trained_models_root_folder = ''

for model_path in Path(trained_models_root_folder).glob('*.keras'):
    model_name = model_path.stem

    model = load_model(model_path.as_posix())
    model_stat = utils.model_stats(model,prep_test_x, test_y)
    print(model_name, model_stat['accuracy_score'])
    print(model_name, model_stat['categorical_accuracy_scores'])
    print()
    

model_trained_by_highest_entropy_sampling_method 0.405
model_trained_by_highest_entropy_sampling_method [0.482 0.672 0.313 0.01  0.418 0.502 0.653 0.184 0.641 0.175]

baseline_model_trained_misclassed_worstclass_highprob 0.3952
baseline_model_trained_misclassed_worstclass_highprob [0.54  0.625 0.122 0.211 0.556 0.259 0.522 0.289 0.603 0.225]

model_trained_by_least_margin_sampling_method 0.3986
model_trained_by_least_margin_sampling_method [0.498 0.649 0.377 0.028 0.42  0.429 0.671 0.072 0.77  0.072]

model_trained_by_uncertainty_method 0.3752
model_trained_by_uncertainty_method [0.449 0.74  0.431 0.    0.507 0.425 0.533 0.01  0.644 0.013]

baseline_model_randomly_sampled_pool 0.4111
baseline_model_randomly_sampled_pool [0.608 0.69  0.314 0.022 0.433 0.507 0.559 0.142 0.618 0.218]

model_trained_by_kmeans_clustered_distant_sampling 0.3882
model_trained_by_kmeans_clustered_distant_sampling [0.668 0.682 0.357 0.005 0.386 0.401 0.671 0.088 0.548 0.076]

baseline_model_trained_on_init 0.35

Plot history of networks


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pickle

trained_models_root_folder = ''

for model_path in Path(trained_models_root_folder).glob('*.pickle'):

    history = pickle.load(open(model_path.as_posix(),'rb'))
    print(model_path)
    
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    